In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,7)
plt.style.use('bmh')
from datetime import timedelta
from keras.models import *
from keras.layers import *
from sklearn.preprocessing import *
import seaborn as sns
from keras.callbacks import *
from sklearn.metrics import *

In [ ]:
# Importing Training Set
df = pd.read_csv('C:\\Users\\moham\\OneDrive - De Montfort University\\Final Project\\Master_Thesis\\Dataset\\QBDaily.csv', index_col=0, parse_dates=True)

# df = df[['Gas', 'Electric', 'Water', 'temp', 'dew_point', 'pressure', 'humidity', 'wind_speed']]

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
values = df.values

# normalize features
scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)

In [ ]:
# specify the number of lag hours
n_hours = 6
n_features = 8

In [ ]:
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)
reframed.head()

In [ ]:
# newpath = 'C:\\Users\\moham\\OneDrive - De Montfort University\\Final Project\\Master_Thesis\\Metrics'
# if not os.path.exists(newpath):
#     os.makedirs(newpath)

In [ ]:
# rmse_txt = [0]
# mse_txt = [0]
# mae_txt = [0]
# rs_txt = [0]

# folds = []

# r_lista = []

# trial = [24, 96, 168]

# for i in trial:
#     folds.append(i)

#     # split into train and test sets
#     values = reframed.values
#     n_train_hours = 365 * i
#     train = values[:n_train_hours, :]
#     test = values[n_train_hours:, :]

#     # split into input and outputs
#     train_X, train_y = train[:, :-1], train[:, -1]
#     test_X, test_y = test[:, :-1], test[:, -1]

#     # reshape input to be 3D [samples, timesteps, features]
#     train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
#     test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))


#     # design network
#     model = Sequential()
#     model.add(LSTM(50, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
#     model.add(Dense(1))

# # Compile
#     es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
#     loss = 'mean_squared_error'
#     model.compile(loss=loss, optimizer='adam')

#     # fit network
#     history = model.fit(train_X, train_y, epochs=50, 
#                         validation_data=(test_X, test_y), 
#                         verbose=0, shuffle=False, callbacks=[es])

#     # plot history
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.legend(['train', 'test'], loc='upper right')
#     plt.xticks(rotation='horizontal')
#     plt.yticks(rotation='horizontal')
#     plt.savefig(newpath + '\\Loss_' + loss + '_Fold_' + str(i) + '.PNG')

#     # make a prediction
#     yhat = model.predict(test_X)
#     test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))

#     # invert scaling for forecast
#     inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
#     inv_yhat = scaler.inverse_transform(inv_yhat)
#     inv_yhat = inv_yhat[:,0]

#     # invert scaling for actual
#     test_y = test_y.reshape((len(test_y), 1))
#     inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
#     inv_y = scaler.inverse_transform(inv_y)
#     inv_y = inv_y[:,0]

#     # plt.plot(inv_y.flatten(), marker='.', label='Actual Data')
#     # plt.plot(inv_yhat.flatten(), marker='v', color='r', label='Predicted Data')
#     # plt.title('Predicted vs Testing', fontweight='bold')
#     # plt.legend(shadow=True)
#     # plt.xticks(rotation='horizontal')
#     # plt.yticks(rotation='horizontal')
#     # plt.xlabel('Data points',fontsize=18, ha='center')
#     # plt.ylabel('kW',fontsize=18, ha='center')
#     # plt.title('Energy Consumption Prediction Validation Data',fontsize=25, ha='center')
#     # plt.tick_params(labelsize=18)
#     # plt.savefig(newpath+ '\\Validation_' + loss + '_Fold_' + str(i) + '.PNG')


#     # calculate RMSE
#     rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
#     mse = mean_squared_error(inv_y, inv_yhat)
#     mae = mean_absolute_error(inv_y, inv_yhat)
#     rs = r2_score(inv_y, inv_yhat)*100

#     r_lista.append([round(rmse, 3), round(mse, 3), round(mae, 3), round(rs, 3)])

#     print('Good Model','RMSE:', round(rmse, 3), 'R^2:', round(rs, 3), 'MAE:', round(mae, 3), 'MSE:', round(mse, 3))

# rmse_txt[0] = r_lista[0][0]
# mse_txt[0]  = r_lista[0][1]
# mae_txt[0]  = r_lista[0][2]
# rs_txt[0]   = r_lista[0][3]

# for i in range(1,len(r_lista)):
#     rmse_txt.append(r_lista[i][0])
#     mse_txt.append(r_lista[i][1])
#     mae_txt.append(r_lista[i][2])
#     rs_txt.append(r_lista[i][3])

# rmse_avg = sum(rmse_txt)/len(r_lista)
# mse_avg = sum(mse_txt)/len(r_lista)
# mae_avg = sum(mae_txt)/len(r_lista)
# rs_avg = sum(rs_txt)/len(r_lista)

# text = newpath + '.txt'
# f= open(text,"w+")
# f.write(str(['RMSE AVG','MSE AVG','MAE AVG','R2 AVG']))
# f.write('\n')
# f.write(str([rmse_avg, mse_avg, mae_avg, rs_avg]))
# f.write('\n')
# f.write('\n')
# f.write(str(['RMSE: ', rmse_txt[:]]))
# f.write('\n')
# f.write(str(['MSE: ', mse_txt[:]]))
# f.write('\n')
# f.write(str(['MAE: ', mae_txt[:]]))
# f.write('\n')
# f.write(str(['R2: ', rs_txt[:]]))
# f.write('\n')
# f.close()

In [ ]:
# split into train and test sets
values = reframed.values
n_train_hours = 365 * 5
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
# model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
# model.add(LSTM(50, activation='relu', return_sequences=True))
# model.add(LSTM(50, activation='relu', return_sequences=True))
# model.add(LSTM(50, activation='relu', return_sequences=True))
# model.add(LSTM(50, activation='relu', return_sequences=True))
# model.add(LSTM(50, activation='relu', return_sequences=True))
# model.add(LSTM(50, activation='relu', return_sequences=False))
# # model.add(Dropout(0.01))
# model.add(Dense(1))
# model.compile(loss='mse', optimizer='adam')

model = Sequential()
model.add(Bidirectional(LSTM(units=50, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True)))
model.add(Bidirectional(LSTM(units=50, activation='relu', return_sequences=True)))
model.add(Bidirectional(LSTM(units=50, activation='relu', return_sequences=True)))
model.add(Bidirectional(LSTM(units=50, activation='relu', return_sequences=True)))
model.add(Bidirectional(LSTM(units=50, activation='relu', return_sequences=True)))
model.add(Bidirectional(LSTM(units=50, activation='relu', return_sequences=True)))
model.add(Bidirectional(LSTM(units=50, activation='relu')))
# model.add(Dropout(rate=0.2))
model.add(Dense(units=1))

# Compiling the Neural Network
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
%%time
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=50, verbose=1)

# fit network
history = model.fit(train_X, train_y, epochs=200, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[es])

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import *

# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], n_hours*n_features))

# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, -7:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, -7:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]

# calculate RMSE
# rmse = np.sqrt(mean_squared_error(test_y, yhat))
# mse = mean_squared_error(test_y, yhat)
# mae = mean_absolute_error(test_y, yhat)
# r2 = r2_score(test_y, yhat)


rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
mse = mean_squared_error(inv_y, inv_yhat)
mae = mean_absolute_error(inv_y, inv_yhat)
r2 = r2_score(inv_y, inv_yhat)

print('Test RMSE: %.3f' % rmse)
print('Test MSE: %.3f' % mse)
print('Test MAE: %.3f' % mae)
print('Test R2: %.3f' % r2)

In [ ]:
plt.plot(inv_y.flatten(), marker='.', label='Actual Data')
plt.plot(inv_yhat.flatten(), marker='v', color='r', label='Predicted Data')
plt.title('Predicted vs Testing', fontweight='bold')
plt.legend(shadow=True);